In [1]:
#Imports
try:
    import html5lib
    import re
    import urllib
    import time
    import pandas as pd

    from DrissionPage import ChromiumPage
    from bs4 import BeautifulSoup
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.common.by import By
    from fake_useragent import UserAgent
    
except ImportError as e:
    print(f"IMPORT ERROR -> {e}")

In [2]:
#University Type (Wikipedia)
def getType(university, soup):
    try:        
        infoTable = soup.find('table', {'class': 'infobox'})
        if infoTable:
            for row in infoTable.find_all('tr'):
                header = row.find('th')
                if header and 'Type' in header.text:
                    locationRow = row
                    
                    if locationRow:
                        locationTd = locationRow.find('td')
                        
                        if locationTd:
                            uniTypeText = locationTd.text.strip()
                            uniTypeText = uniTypeText.lower()
    
                            if "private" in uniTypeText:
                                return "Private"
                            elif "public" in uniTypeText:
                                return "Public"
                            else:
                                return uniTypeText
                            break
                            return
                        else:
                            print(f"Location td not found for {university}")
                            break
                            return
                            
                    else:
                        print(f"Location row not found for {university}")
                        break
                        return
        else:
            print(f"Info table not found for {university}")
            return
                
    except Exception as e:
        print(f"ERROR WHILE FETCHING UNIVERSITY TYPE -> {e}")
        return

In [3]:
#University Location (Wikipedia)
def getLocation(university, soup):
    try:
        infoTable = soup.find('table', {'class': 'infobox vcard'})
        if infoTable:
            for row in infoTable.find_all('tr'):
                header = row.find('th')
                if header and 'Location' in header.text:
                    locationRow = row
                    if locationRow:
                        locationTd = locationRow.find('td')
                        
                        if locationTd:
                            uniLocationDiv = locationTd.find('div')
                            return uniLocationDiv.text.strip()
                            break
                        else:
                            print(f"Location td not found for {university}")
                            break
                    
        else:
            print(f"Info table not found for {university}")
            return
            
    except Exception as e:
        print(f"ERROR WHILE FETCHING UNIVERSITY LOCATION -> {e}")
        return

In [4]:
#University Summary (Wikipedia)
def getSummary(university, soup):
    try:        
        infoTable = soup.find('table', {'class': 'infobox vcard'})
        return infoTable.find_next_sibling().text
        
    except Exception as e:
        print(f"UNEXPECTED ERROR -> {e}")
        return

In [5]:
#University Website (Wikipedia)
def getWebsite(university, soup):
    try:        
        infoTable = soup.find('table', {'class': 'infobox vcard'})
        if infoTable:
            for row in infoTable.find_all('tr'):
                header = row.find('th')
    
                if header and "Website" in header.text:
                    locationRow = row
    
                    if locationRow:
                        return locationRow.find('a')['href']
                        break
                    else:
                        print(f"Location row not found for {university}")
                        return
    
        else:
            print(f"infoTable not found for {university}")
            return
                
    except Exception as e:
        print(f"UNEXPECTED ERROR -> {e}")
        return

In [6]:
#Established (Wikipedia)
def getEstablished(university, soup):
    try:       
        infoTable = soup.find('table', {'class': 'infobox vcard'})
        if infoTable:
            for row in infoTable.find_all('tr'):
                header = row.find('th')
                if header and 'Established' in header.text:
                    locationRow = row
                    
                    if locationRow:
                        locationTd = locationRow.find('td')
                        
                        if locationTd:
                            established = locationTd.text.strip()
                            established = established.lower()
    
                            result = re.search(r"\d{4}", established)
                            if result:
                                return result.group(0)
                                
                            else:
                                print(f"Established not found for {university}")
                                return
                        else:
                            print(f"Location td not found for {university}")
                            break
                            return
                            
                    else:
                        print(f"Location row not found for {university}")
                        break
                        return
        else:
            print(f"Info table not found for {university}")
            return
                
    except Exception as e:
        print(f"ERROR WHILE FETCHING UNIVERSITY TYPE -> {e}")
        return

In [7]:
#University Enrollment (Wikipedia)
def getEnrollment(university, soup):
    try:       
        infoTable = soup.find('table', {'class': 'infobox vcard'})
        if infoTable:
            for row in infoTable.find_all('tr'):
                header = row.find('th')
                if header and 'Students' in header.text:
                    locationRow = row
                    
                    if locationRow:
                        locationTd = locationRow.find('td')
                        
                        if locationTd:
                            locationTdText = locationTd.text
                            result = re.search(r"^\d+(?:,\d{3})*", locationTdText)
    
                            if result:
                                return result.group(0)
                            else:
                                print(locationTdText)
                            break
                            return
                        else:
                            print(f"Location td not found for {university}")
                            break
                            return
                            
                    else:
                        print(f"Location row not found for {university}")
                        break
                        return
        else:
            print(f"Info table not found for {university}")
            return
            
    except Exception as e:
        print(f"ERROR WHILE FETCHING UNIVERSITY LOCATION -> {e}")
        return

In [8]:
#Application Link (Google Search)
def getApplicationLink(unviersity):
    try:
        query = urllib.parse.urlencode({"q": r"{} international graduate apply".format(university)})
        
        driver = ChromiumPage()
        res = driver.get(r"https://www.google.com/search?{}".format(query))
        page_source = driver.html
        
        soup = BeautifulSoup(page_source, 'html.parser')
        
        applicationLinkElement = soup.find('h3',string=re.compile(r"Apply|Admission|Application")).find_parent('a')
        result = applicationLinkElement['href']
        
        time.sleep(0.5)
        
        if result:
            return result
        else:
            print(f"Application Link for {university} not found")
            return

    except Exception as e:
        print(f"ERROR WHILE FETCHING UNIVERSITY APPLICATION LINK -> {e}")
        return

In [9]:
#Funding Link (Google Search)
def getFundingLink(university):
    try:
        query = urllib.parse.urlencode({"q": r"{} international graduate financial aid".format(university)})
        
        driver = ChromiumPage()
        res = driver.get(r"https://www.google.com/search?{}".format(query))
        page_source = driver.html
        
        soup = BeautifulSoup(page_source, 'html.parser')
        
        fundingLinkElement = soup.find('h3',string=re.compile(r"Funding|Financial|Aid|Scholarship")).find_parent('a')
        result = fundingLinkElement['href']
        
        time.sleep(3)
        
        if result:
            return result
        else:
            print(f"Funding Link for {university} not found")
            return
    except Exception as e:
        print(f"ERROR WHILE FETCHING UNIVERSITY FUNDING LINK -> {e}")
        return

In [10]:
#Acceptance Rate (Google Search)
def getAcceptanceRate(university):
    try:
        query = urllib.parse.urlencode({"q": r"{}".format(university)})
        
        driver = ChromiumPage()
        res = driver.get(r"https://www.google.com/search?{}".format(query))
        page_source = driver.html
        
        soup = BeautifulSoup(page_source, 'html.parser')
        
        h2Element = soup.find('h2', text='Complementary Results')
        
        time.sleep(0.5)
        
        if h2Element:
            h2Parent = h2Element.find_parent('div')
        
            if h2Parent:
                mainDiv = h2Parent.find_all('div', class_='rVusze')
                for item in mainDiv:
                    if 'Acceptance rate' in item.text:
                        itemText = item.text
                        return itemText[(itemText.find(":") + 2):(itemText.find("%") + 1)]
                        
            else:
                print(f'Complementary Results found, but further elements not found for {university}')
                return
        
        else:
            print(f"Complementary Results for {university} not found")
            return
            
    except Exception as e:
        print(f"ERROR WHILE FETCHING UNIVERSITY ACCEPTANCE RATE -> {e}")
        return

In [29]:
#Pre-processing for webdriver
query = None
# universityList = [
#     "Massachusetts Institute of Technology (MIT)",
#     "Harvard University",
#     "Stanford University",
#     "California Institute of Technology (Caltech)",
#     "University of Pennsylvania",
#     "University of California, Berkeley (UCB)",
#     "Cornell University",
#     "University of Chicago",
#     "Princeton University",
#     "Yale University",
#     "Johns Hopkins University",
#     "Columbia University",
#     "University of California, Los Angeles (UCLA)",
#     "New York University (NYU)",
#     "University of Michigan-Ann Arbor",
#     "Northwestern University",
#     "Carnegie Mellon University",
#     "Duke University",
#     "University of Texas at Austin",
#     "University of Illinois at Urbana-Champaign"
# ]
# universityList = [
#     "Massachusetts Institute of Technology (MIT)",
#     "Imperial College London",
#     "University of Oxford",
#     "Harvard University",
#     "University of Cambridge",
#     "Stanford University",
#     "ETH Zurich - Swiss Federal Institute of Technology",
#     "National University of Singapore (NUS)",
#     "University College London",
#     "California Institute of Technology (Caltech)",
#     "University of Pennsylvania",
#     "University of California, Berkeley (UCB)",
#     "The University of Melbourne",
#     "Peking University",
#     "Nanyang Technological University, Singapore (NTU)",
#     "Cornell University",
#     "The University of Hong Kong",
#     "The University of Sydney",
#     "The University of New South Wales (UNSW Sydney)",
#     "Tsinghua University",
#     "University of Chicago",
#     "Princeton University",
#     "Yale University",
#     "Université PSL",
#     "University of Toronto",
#     "Swiss Federal Technology Institute of Lausanne",
#     "The University of Edinburgh",
#     "Technical University of Munich",
#     "McGill University",
#     "The Australian National University",
#     "Seoul National University",
#     "Johns Hopkins University",
#     "The University of Tokyo",
#     "Columbia University",
#     "The University of Manchester",
#     "The Chinese University of Hong Kong (CUHK)",
#     "Monash University",
#     "University of British Columbia",
#     "Fudan University",
#     "King's College London",
#     "The University of Queensland",
#     "University of California, Los Angeles (UCLA)",
#     "New York University (NYU)",
#     "University of Michigan-Ann Arbor",
#     "Shanghai Jiao Tong University",
#     "Polytechnic Institute of Paris", #"Institut Polytechnique de Paris"
#     "The Hong Kong University of Science and Technology",
#     "Zhejiang University",
#     "Delft University of Technology",
#     "Kyoto University",
#     "Northwestern University",
#     "The London School of Economics and Political Science (LSE)",
#     "KAIST - Korea Advanced Institute of Science & Technology",
#     "University of Bristol",
#     "University of Amsterdam",
#     "Yonsei University",
#     "The Hong Kong Polytechnic University",
#     "Carnegie Mellon University",
#     "Ludwig-Maximilians-Universität München",
#     "Universiti Malaya (UM)",
#     "Duke University",
#     "City University of Hong Kong",
#     "KU Leuven",
#     "Sorbonne University",
#     "The University of Auckland",
#     "University of Texas at Austin",
#     "Korea University",
#     "National Taiwan University (NTU)",
#     "The University of Warwick",
#     "University of Illinois at Urbana-Champaign",
#     "Universidad de Buenos Aires (UBA)",
#     "University of California, San Diego (UCSD)",
#     "Paris-Saclay University", #"Université Paris-Saclay"
#     "KTH Royal Institute of Technology",
#     "Lund University",
#     "University of Washington",
#     "The University of Western Australia",
#     "University of Glasgow",
#     "Brown University",
#     "University of Birmingham",
#     "University of Southampton",
#     "The University of Adelaide",
#     "University of Leeds",
#     "Ruprecht-Karls-Universität Heidelberg",
#     "Tokyo Institute of Technology (Tokyo Tech)",
#     "Osaka University",
#     "Trinity College Dublin",
#     "The University of Dublin",
#     "University of Technology Sydney",
#     "Durham University",
#     "Pennsylvania State University",
#     "Purdue University",
#     "University of São Paulo", #"Universidade de São Paulo"
#     "Pontifical Catholic University of Chile", #"Pontificia Universidad Católica de Chile (UC)"
#     "Lomonosov Moscow State University",
#     "National Autonomous University of Mexico", #"Universidad Nacional Autónoma de México (UNAM)"
#     "University of Alberta",
#     "Free University of Berlin", #"Freie Universitaet Berlin"
#     "Pohang University of Science And Technology (POSTTECH)",
#     "RWTH Aachen University",
#     "University of Copenhagen"
# ]
universityList = [
    "Massachusetts Institute of Technology (MIT)",
    "Harvard University",
    "Stanford University",
    "California Institute of Technology (Caltech)",
    "University of Pennsylvania",
    "Princeton University",
    "University of California, Berkeley (UCB)",
    "Yale University",
    "Cornell University",
    "University of Chicago",
    "Columbia University",
    "Johns Hopkins University",
    "Northwestern University",
    "Duke University",
    "University of Michigan - Ann Arbor",
    "New York University (NYU)",
    "Carnegie Mellon University",
    "University of California, Los Angeles (UCLA)",
    "University of Texas at Austin",
    "Rice University",
    "University of Illinois at Urbana-Champaign",
    "University of California, San Diego (UCSD)",
    "University of Washington",
    "Brown University",
    "Purdue University",
    "Pennsylvania State University",
    "Boston University",
    "Georgia Institute of Technology",
    "University of Wisconsin-Madison",
    "University of Southern California",
    "University of California, Davis",
    "Michigan State University",
    "Texas A&M University",
    "University of North Carolina at Chapel Hill",
    "Washington University in St. Louis",
    "University of California, Santa Barbara",
    "Emory University",
    "Arizona State University",
    "University of Minnesota - Twin Cities",
    "Ohio State University",
    "University of Florida",
    "University of Maryland at College Park",
    "University of Rochester",
    "Dartmouth College",
    "Vanderbilt University",
    "Case Western Reserve University",
    "University of Massachusetts, Amherst",
    "University of Pittsburgh",
    "University of Arizona",
    "University of Virginia"
]
options = Options()
ua = UserAgent()
user_agent = ua.random

options.add_argument(f'--user-agent={user_agent}')

In [30]:
#Websriver for Wikipedia Searches
def wikipediaSearch(university):
    query = urllib.parse.urlencode({"q": r"{} wikipedia".format(university)})
    
    driver = ChromiumPage()
    res = driver.get(r"https://www.google.com/search?{}".format(query))
    page_source = driver.html
    
    time.sleep(0.5)
    firstTab = driver.ele('tag:h3')
    firstTab.click()
    
    time.sleep(0.5)
    page_source = driver.html
    soup = BeautifulSoup(page_source, "html.parser")

    return soup

In [31]:
#Extraction
finalData = []
iterationTimings = []
dataHeaders = ["Name","Location","Type","Summary","Website","Established","Enrollment","Application Link","Funding Link","Acceptance Rate"]

for university in universityList:
    startTime = time.time()
    universityData = []

    soup = wikipediaSearch(university)
    
    universityData.clear()
    universityData.append(university)
    universityData.append(getLocation(university, soup))
    universityData.append(getType(university, soup))
    universityData.append(getSummary(university, soup))
    universityData.append(getWebsite(university, soup))
    universityData.append(getEstablished(university, soup))
    universityData.append(getEnrollment(university, soup))
    universityData.append(getApplicationLink(university))
    universityData.append(getFundingLink(university))
    universityData.append(getAcceptanceRate(university))
    
    finalData.append(universityData)
    
    endTime = time.time()
    
    timeTaken = endTime - startTime
    print(timeTaken)
    print(f"Data extracted for {university}")
    print("-"*10)

    iterationTimings.append(timeTaken)

df = pd.DataFrame(finalData, columns=dataHeaders)
df.to_csv("Top50_USA.csv", index=False)

C:\Users\Amandeep\AppData\Local\Temp\ipykernel_34392\2719239113.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  h2Element = soup.find('h2', text='Complementary Results')


13.133927583694458
Data extracted for Massachusetts Institute of Technology (MIT)
----------
12.06949496269226
Data extracted for Harvard University
----------
12.571660041809082
Data extracted for Stanford University
----------
11.231377840042114
Data extracted for California Institute of Technology (Caltech)
----------
11.018185138702393
Data extracted for University of Pennsylvania
----------
13.152277708053589
Data extracted for Princeton University
----------
11.419727087020874
Data extracted for University of California, Berkeley (UCB)
----------
12.291146755218506
Data extracted for Yale University
----------
11.372976779937744
Data extracted for Cornell University
----------
11.533077239990234
Data extracted for University of Chicago
----------
12.25087022781372
Data extracted for Columbia University
----------
11.40947961807251
Data extracted for Johns Hopkins University
----------
11.287239074707031
Data extracted for Northwestern University
----------
11.671358823776245
Data

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              50 non-null     object
 1   Location          50 non-null     object
 2   Type              50 non-null     object
 3   Summary           50 non-null     object
 4   Website           49 non-null     object
 5   Established       50 non-null     object
 6   Enrollment        50 non-null     object
 7   Application Link  50 non-null     object
 8   Funding Link      50 non-null     object
 9   Acceptance Rate   28 non-null     object
dtypes: object(10)
memory usage: 4.0+ KB
